In [115]:
import pandas as pd
import numpy as np
from datetime import datetime

In [116]:
def transform_ts_to_datetime(ts):
    return datetime.utcfromtimestamp(ts)

def datetime_to_date(datetime):
    # return datetime.date().strftime('%Y-%m-%d %H:%M:%S')
    return datetime.date().strftime('%Y-%m-%d')

# df['datetime'] = df['ts'].map(transform_ts_to_datetime)
# df['date'] = df['datetime'].map(datetime_to_date)
# df

In [117]:
data = pd.DataFrame({
    'order_id': [1,2,3,4,5],
    'user_id': [1,2,3,4,5],
    'order_time': ['2021-09-12', '2021-09-12', '2021-09-12', '2021-09-13', '2021-09-14'],
    'items': [
        [{
            'ItemID': 1,
            'Price': 100,
            'quantity': 2
        }, {
            'ItemID': 2,
            'Price': 20,
            'quantity': 12
        }, {
            'ItemID': 4,
            'Price': 50,
            'quantity': 2
        }], 
        
        [{
            'ItemID': 1,
            'Price': 100,
            'quantity': 5
        }, {
            'ItemID': 2,
            'Price': 250,
            'quantity': 1
        }], 
        
        [{
            'ItemID': 1,
            'Price': 100,
            'quantity': 1
        }], 
        
        [{
            'ItemID': 1,
            'Price': 100,
            'quantity': 7
        }], 
        
        [{
            'ItemID': 1,
            'Price': 100,
            'quantity': 4
        }] ]
})
data

,order_id,user_id,order_time,items
0,1,1,2021-09-12,"[{'ItemID': 1, 'Price': 100, 'quantity': 2}, {..."
1,2,2,2021-09-12,"[{'ItemID': 1, 'Price': 100, 'quantity': 5}, {..."
2,3,3,2021-09-12,"[{'ItemID': 1, 'Price': 100, 'quantity': 1}]"
3,4,4,2021-09-13,"[{'ItemID': 1, 'Price': 100, 'quantity': 7}]"
4,5,5,2021-09-14,"[{'ItemID': 1, 'Price': 100, 'quantity': 4}]"


In [118]:
categories = pd.DataFrame({
    'ItemID': [1,2,3,4,5],
    'Category': [2,2,1,3,2]
})
categories

,ItemID,Category
0,1,2
1,2,2
2,3,1
3,4,3
4,5,2


In [119]:
data_df = pd.DataFrame(data['items'])
data_df = data_df.explode(column='items')
data_df['ID'] = data_df['items'].apply(lambda x: x['ItemID'])
data_df['Revenue'] = data_df['items'].apply(lambda x: x['Price'] * x['quantity'])
data_df = data_df.drop(columns='items')
data_df

,ID,Revenue
0,1,200
0,2,240
0,4,100
1,1,500
1,2,250
2,1,100
3,1,700
4,1,400


In [120]:
joined = pd.merge(data_df, categories, left_on='ID', right_on='ItemID', how='inner')
joined

,ID,Revenue,ItemID,Category
0,1,200,1,2
1,1,500,1,2
2,1,100,1,2
3,1,700,1,2
4,1,400,1,2
5,2,240,2,2
6,2,250,2,2
7,4,100,4,3


In [121]:
joined.groupby('Category').apply('sum').sort_values('Revenue', ascending=False)

,ID,Revenue,ItemID
Category,,,
2,9,2390,9
3,4,100,4


# Новые данные

In [152]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Function to generate random order data
def generate_order_data(order_id, user_id, date):
    items_count = random.randint(1, 10)
    items = []
    for _ in range(items_count):
        item = {
            'ItemID': random.randint(1, 10),
            'Price': random.randint(10, 500),
            'quantity': random.randint(1, 10)
        }
        items.append(item)
    return {
        'order_id': order_id,
        'user_id': user_id,
        'order_time': date,
        'items': items
    }

# Generate the dataset with orders from September 1 to September 25
start_date = datetime(2021, 9, 1)
end_date = datetime(2021, 9, 25)
date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

data = []
order_id = 1
for date in date_range:
    daily_orders = random.randint(1, 10)  # Random number of orders per day
    for _ in range(daily_orders):
        user_id = random.randint(1, 20)  # Random user ID
        data.append(generate_order_data(order_id, user_id, date.strftime('%Y-%m-%d')))
        order_id += 1

# Convert to DataFrame
df = pd.DataFrame(data)
df.head(3)

,order_id,user_id,order_time,items
0,1,20,2021-09-01,"[{'ItemID': 3, 'Price': 245, 'quantity': 7}, {..."
1,2,7,2021-09-01,"[{'ItemID': 7, 'Price': 259, 'quantity': 3}, {..."
2,3,14,2021-09-01,"[{'ItemID': 6, 'Price': 368, 'quantity': 4}, {..."


In [156]:
# хочу посчитать последний день, в который пользователем совершалась покупка

df = df[['user_id', 'order_time']]
df['max_time'] = df.groupby('user_id')['order_time'].transform('max')
df

,user_id,order_time,max_time
0,20,2021-09-01,2021-09-25
1,7,2021-09-01,2021-09-22
2,14,2021-09-01,2021-09-24
3,3,2021-09-01,2021-09-12
4,3,2021-09-01,2021-09-12
...,...,...,...
116,14,2021-09-24,2021-09-24
117,1,2021-09-24,2021-09-24
118,10,2021-09-25,2021-09-25
119,16,2021-09-25,2021-09-25


In [185]:
# допустим, сегодня 26 сентября 2021. Люди для скидок - это люди, которые, скажем, не были у нас последние три дня

df[pd.Timestamp('2021-09-26') - df['max_time'].apply(lambda x: pd.Timestamp(x)) > pd.Timedelta('3 days 00:00:00')].tail()

,user_id,order_time,max_time
102,15,2021-09-21,2021-09-22
103,15,2021-09-21,2021-09-22
107,15,2021-09-22,2021-09-22
108,7,2021-09-22,2021-09-22
110,7,2021-09-22,2021-09-22


In [174]:
pd.Timestamp('2024-09-26') - pd.Timestamp('2024-09-25') > pd.Timedelta('0 days 00:00:00')

True